# 安全与验证

## 为你的密码加密

用户密码不应该明文保存在数据库中,这样如果数据库登录权限出了问题我们还可以保住用户的密码信息,我们可以使用[`itsdangerous`](https://pythonhosted.org/itsdangerous/)来实现.

我们使用`URLSafeSerializer`这个类来做密码的序列化和反序列化


In [1]:
from itsdangerous import URLSafeSerializer as USer

In [2]:
s1 = USer("secret",salt="salt1")

In [4]:
ss = s1.dumps("8812321")
print ss

Ijg4MTIzMjEi.fATSYUPXhGaeODWWVyYxGq7cNsE


In [5]:
s1.loads(ss)

u'8812321'

我们来改造我们的用户服务

In [19]:
%%writefile codes/c7/app.py
# --*-- coding:utf-8 --*--
from flask import Flask,request,jsonify
from flask_restful import Api,Resource
import json
from itsdangerous import URLSafeSerializer as USer

app = Flask(__name__)
api = Api(app)

import os

from peewee import SqliteDatabase
db = SqliteDatabase(os.path.abspath("peewee_db.sqlite"))
from peewee import Model,CharField,DateField,BooleanField,ForeignKeyField
class User(Model):
    name = CharField()
    password = CharField()
    type = CharField(default="local")
    
    SER = USer("secret",salt="salt1")
    def __str__(self):
        return "<USER: id-{self.id}--name-{self.name}>".format(self=self)
    def __repr__(self):
        return self.__str__()
    
    def verify_password(self,target):
        return True if self.SER.loads(self.password) == target else False
    @staticmethod
    def hash_password(org):
        return User.SER.dumps(org)
    class Meta:
        database = db # 确定使用的是哪个db,方便多种数据库配合使用
        
db.connect() 
try:
    db.create_tables([User])
except:
    print " * db is ready"
@app.errorhandler(404)
def page_not_found(error):
    return jsonify({"message":"404 not found","code":404}), 404

class UsersAPI(Resource):
    
    def get(self):
        result = [user.__str__() for user in list(User.select())]
        return {"result":result}

    def post(self):
        data = json.loads(request.data)
        user = User(name = data.get("name"),password =User.hash_password(data.get("password"))).save()
        return jsonify({"result":"save {name} done!".format(name=data.get("name"))})
    
class UserAPI(Resource):

    def get(self,user_id):
        
        user = User.get(User.id==user_id)
        if not user:
            abort(404)
            
        result = {"name":user.name,"password":user.password,"type":user.type}
        return jsonify({"result":result})
    
  
    def put(self,user_id):
        data = request.args
        user = User.get(User.id==user_id)
        if request.args.get("name"):
            user.name=data.get("name")
        if request.args.get("password"):
            user.password=User.hash_password(data.get("password"))
        user.save()
        
        return jsonify({"result":"update {name} done!".format(name=user_id)})  

    def delete(self,user_id):
        
        User.get(User.id==user_id).delete_instance() 
        return jsonify({"result":"delete {id} done!".format(id=user_id)})  
    
    
api.add_resource(UsersAPI, '/users/')        
api.add_resource(UserAPI, '/user/<int:user_id>')  

if __name__ == '__main__':
    app.run(debug=True)

Overwriting codes/c7/app.py


In [9]:
import requests
import json

In [20]:
requests.get("http://127.0.0.1:5000/users/").json()

{u'result': [u'<USER: id-1--name-niuniu>']}

In [16]:
requests.post("http://127.0.0.1:5000/users/",data = json.dumps({"name":"niuniu","password":"qwe"}),
              headers={'content-type': 'application/json'}).json()

{u'result': u'save niuniu done!'}

In [21]:
requests.get("http://127.0.0.1:5000/user/1").json()

{u'result': {u'name': u'niuniu',
  u'password': u'InF3ZSI.3nj9apTCzTpZbqeMv5qiBEPCGPE',
  u'type': u'local'}}

我们的API服务的安全验证算是完成了,这个方法可以让你的用户登录信息不会暴露在外面,同时时间戳也让风险降低了许多.使用的时候步骤是这样的:

先去/verify路径下提交登录信息,它会返回一个包含用户信息的token,之后每次操作的时候提交验证数据时只要第一位的user内放入token,并第二位的password放入空字符串即可,而如果password位有内容则说明是使用传统的用户密码登录.
无论哪种方式登录,auth.me都将保存登录的用户名

当然了一般来说我们需要使用数据库,这个例子中没用数据库,但改造起来很简单,有兴趣的可以尝试自己写写~

# 总结

用到的模块:

包|作用
---|---
flask|flask web框架
flask-restful|flask的restApi框架
flask_httpauth|flask的http验证框架
itsdangerous|加密及序列化模块
datetime | 时间操作模块